# Crear Target

## Ejercicio 1:
Cree el archivo `competencia_01.csv`, usando el `competencia_01_crudo.csv` con una variable adicional llamada `clase_ternaria`, que contenga las categorías **CONTINUA**, **BAJA+1** y **BAJA+2** 

### Ayudita 

Para practicar el muy útil y necesario lenguaje **SQL**, vamos a utilizar una base de datos **OLAP** llamada **DuckDB**. 

La documentación la puede encontrar [aquí](https://duckdb.org/docs/archive/0.8.1/sql/introduction)
Procedemos a instalarla, esto se debe ejecutar una sola vez

In [1]:
%%bash 
pip install duckdb
pip install jupysql
pip install duckdb-engine
pip install pandas

Configuramos el entorno de ejecución. Si ya tiene todo instalado, solo necesita ejecutar esta celda para empezar a usar **duckdb** 

In [2]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:

There's a new jupysql version available (0.10.1), you're running 0.10.0. To upgrade: pip install jupysql --upgrade


Y ya podemos usar **SQL** dentro de una notebook!

In [3]:
%%sql
select 'hola mundo' 

,'hola mundo'
0,hola mundo


Para cargar el archivo `.csv` a una tabla:

In [4]:
%%sql
create or replace table competencia_01 as 
select 
    *
from read_csv_auto('/home/tomi/Escritorio/Maestria/2 - DMEyF/datasets/competencia_01_crudo.csv')

,Success


In [5]:
%%sql
SELECT COUNT(*) AS total_filas
FROM competencia_01;

,total_filas
0,491063


Hagamos unas queries básicas para comprobar que todo esta funcionando bien.

In [30]:
%sql select highest_foto_mes from competencia_01

RuntimeError: If using snippets, you may pass the --with argument explicitly.
For more details please refer: https://jupysql.ploomber.io/en/latest/compose.html#with-argument


Original error message from DB driver:
(duckdb.BinderException) Binder Error: Referenced column "highest_foto_mes" not found in FROM clause!
Candidate bindings: "competencia_01.foto_mes"
LINE 1: select highest_foto_mes from competencia_01
               ^
[SQL: select highest_foto_mes from competencia_01]
(Background on this error at: https://sqlalche.me/e/20/f405)

If you need help solving this issue, send us a message: https://ploomber.io/community


In [25]:
%sql select * from competencia_01 limit 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,29183981,202103,1,0,0,50,197,14468.81,125765.29,2389.82,...,0.0,7,0.00,-114954.00,0.00,5938,101050.66,68,0,71811.06
1,29184630,202103,1,0,0,59,322,11901.57,74158.93,18750.68,...,0.0,1,0.00,-40330.15,17.59,4089,26834.09,7,0,3894.36
2,29185433,202103,1,0,0,68,268,847.15,21672.47,481.62,...,0.0,21,4692.00,-1173.00,0.00,7829,1651.36,3,0,1560.09
3,29185587,202103,1,0,0,79,322,4976.94,47735.98,1839.31,...,0.0,1,0.00,-15988.67,0.00,7580,30025.29,11,0,1700.85
4,29185646,202103,1,0,0,60,257,2860.45,37800.71,4035.40,...,0.0,21,380616.14,-97383.25,0.00,7827,359610.70,31,0,15600.90


In [6]:
%%sql
select 
    foto_mes
    , count(*) as cantidad
from competencia_01
group by foto_mes

,foto_mes,cantidad
0,202103,163324
1,202104,163637
2,202105,164102


In [38]:
%%sql
create or replace table meses AS
SELECT DISTINCT foto_mes
FROM competencia_01;

,Success


In [24]:
%sql select * from meses 

RuntimeError: If using snippets, you may pass the --with argument explicitly.
For more details please refer: https://jupysql.ploomber.io/en/latest/compose.html#with-argument


Original error message from DB driver:
(duckdb.CatalogException) Catalog Error: Table with name meses does not exist!
Did you mean "temp.information_schema.tables"?
LINE 1: select * from meses
                      ^
[SQL: select * from meses]
(Background on this error at: https://sqlalche.me/e/20/f405)

If you need help solving this issue, send us a message: https://ploomber.io/community


In [6]:
%%sql
CREATE TEMPORARY TABLE consulta_sql AS
SELECT 'SELECT cliente_id, ' || 
       LISTAGG('MAX(CASE WHEN mes = ''' || mes || ''' THEN 1 ELSE 0 END) AS ' || mes, ', ') 
       WITHIN GROUP (ORDER BY mes) || 
       ' FROM tu_tabla_original GROUP BY cliente_id;' AS sql
FROM meses_disponibles;

RuntimeError: (duckdb.ParserException) Parser Error: Unknown ordered aggregate "LISTAGG".
[SQL: CREATE TEMPORARY TABLE consulta_sql AS
SELECT 'SELECT cliente_id, ' || 
       LISTAGG('MAX(CASE WHEN mes = ''' || mes || ''' THEN 1 ELSE 0 END) AS ' || mes, ', ') 
       WITHIN GROUP (ORDER BY mes) || 
       ' FROM tu_tabla_original GROUP BY cliente_id;' AS sql
FROM meses_disponibles;]
(Background on this error at: https://sqlalche.me/e/20/f405)
If you need help solving this issue, send us a message: https://ploomber.io/community


Perfecto, ahora cree una nueva tabla con la variable adicional que se le pide. Algunas funciones que le pueden ser útiles:  [where](https://duckdb.org/docs/sql/query_syntax/where), [left join](https://duckdb.org/docs/sql/query_syntax/from), [case statement](https://duckdb.org/docs/sql/expressions/case)



In [7]:
%%sql
SELECT
    numero_de_cliente,
    MAX(CASE WHEN foto_mes = 202103 THEN 1 ELSE 0 END) AS Enero,
    MAX(CASE WHEN foto_mes = 202104 THEN 1 ELSE 0 END) AS Febrero,
    MAX(CASE WHEN foto_mes = 202105 THEN 1 ELSE 0 END) AS Marzo,
    -- Repite esto para los demás meses.
FROM
    competencia_01
GROUP BY
    numero_de_cliente;


,numero_de_cliente,Enero,Febrero,Marzo
0,29185646,1,1,1
1,29188883,1,1,1
2,29198669,1,1,1
3,29202529,1,1,1
4,29205501,1,1,1
...,...,...,...,...
166115,54175890,0,0,1
166116,58674153,0,0,1
166117,57137632,0,0,1
166118,60030971,0,0,1


In [61]:
%%sql
SELECT
    numero_de_cliente,
    LIST(((foto_mes//100-2021)*12+foto_mes%100) order by foto_mes asc) as mes,
FROM
    competencia_01
GROUP BY    
    numero_de_cliente;

,numero_de_cliente,mes
0,29185646,"[3, 4, 5]"
1,29188883,"[3, 4, 5]"
2,29198669,"[3, 4, 5]"
3,29202529,"[3, 4, 5]"
4,29205501,"[3, 4, 5]"
...,...,...
166115,54175890,[5]
166116,58674153,[5]
166117,57137632,[5]
166118,60030971,[5]


In [100]:
%%sql
SELECT numero_de_cliente,foto_mes
FROM competencia_01
ORDER BY numero_de_cliente, foto_mes;

,numero_de_cliente,foto_mes
0,29183981,202103
1,29183981,202104
2,29183981,202105
3,29184630,202103
4,29184630,202104
...,...,...
491058,186790659,202105
491059,186792315,202105
491060,186795526,202105
491061,186798379,202105


In [22]:
%%sql
ROLLBACK

,Success


In [8]:
%%sql
create or replace table proceso AS
SELECT numero_de_cliente,foto_mes,meses_futuro_activo,
CASE WHEN meses_futuro_activo > 2 THEN 'CONTINUA'
    ELSE ('BAJA+' || meses_futuro_activo)
    END as clase_ternaria
FROM(
    SELECT numero_de_cliente,foto_mes,
    SUM(aux) OVER (
        PARTITION BY "numero_de_cliente"
        ORDER BY numero_de_cliente, foto_mes
        ROWS BETWEEN 0 PRECEDING
            AND 2 FOLLOWING) AS meses_futuro_activo
    FROM 
        (SELECT 
            numero_de_cliente,
            foto_mes,
            CASE 
                WHEN foto_mes = 202105 THEN 3 
                WHEN foto_mes = 202104 THEN 2 
                ELSE 1 END AS aux
        FROM competencia_01
        ORDER BY numero_de_cliente, foto_mes)
    ORDER BY numero_de_cliente, foto_mes
);

,Success


In [128]:
%%sql
SELECT numero_de_cliente,foto_mes,meses_futuro_activo,
CASE WHEN meses_futuro_activo > 2 THEN 'CONTINUA'
    ELSE ('BAJA+' || meses_futuro_activo)
    END as clase_ternaria
FROM(
    SELECT numero_de_cliente,foto_mes,
    SUM(aux) OVER (
        PARTITION BY "numero_de_cliente"
        ORDER BY numero_de_cliente, foto_mes
        ROWS BETWEEN 0 PRECEDING
            AND 2 FOLLOWING) AS meses_futuro_activo
    FROM 
        (SELECT 
            numero_de_cliente,
            foto_mes,
            (foto_mes//100-2021)*12+foto_mes%100 as mes
        FROM competencia_01
        ORDER BY numero_de_cliente, foto_mes)
    ORDER BY numero_de_cliente, foto_mes
);

,numero_de_cliente,foto_mes,meses_futuro_activo,clase_ternaria
0,29183981,202103,6.0,CONTINUA
1,29183981,202104,5.0,CONTINUA
2,29183981,202105,3.0,CONTINUA
3,29184630,202103,6.0,CONTINUA
4,29184630,202104,5.0,CONTINUA
...,...,...,...,...
491058,186790659,202105,3.0,CONTINUA
491059,186792315,202105,3.0,CONTINUA
491060,186795526,202105,3.0,CONTINUA
491061,186798379,202105,3.0,CONTINUA


In [147]:
%%sql
    SELECT numero_de_cliente,foto_mes,mes,
    lead(mes,1,-1) OVER (PARTITION BY "numero_de_cliente" ORDER BY numero_de_cliente, mes) AS 'mes+1',
    lead(mes,2,-1) OVER (PARTITION BY "numero_de_cliente" ORDER BY numero_de_cliente, mes) AS 'mes+2',  
    FROM 
        (SELECT 
            numero_de_cliente,
            foto_mes,
            (foto_mes//100-2021)*12+foto_mes%100 as mes
        FROM competencia_01
        ORDER BY numero_de_cliente, foto_mes)
    ORDER BY numero_de_cliente, foto_mes;

,numero_de_cliente,foto_mes,mes,mes+1,mes+2
0,29183981,202103,3,4,5
1,29183981,202104,4,5,-1
2,29183981,202105,5,-1,-1
3,29184630,202103,3,4,5
4,29184630,202104,4,5,-1
...,...,...,...,...,...
491058,186790659,202105,5,-1,-1
491059,186792315,202105,5,-1,-1
491060,186795526,202105,5,-1,-1
491061,186798379,202105,5,-1,-1


In [11]:
%%sql
ROLLBACK

,Success


In [13]:
%%sql
create or replace table proceso AS
SELECT *,
CASE
    -- Condiciones de borde:
    WHEN ("mes+1"=0) THEN 'CONTINUA'
    WHEN ("mes+2"=0) THEN 'CONTINUA'
    -- Si mes siguiente no es consecutivo o no existe:
    WHEN ("mes"+1 != "mes+1") THEN 'BAJA+1'
    WHEN ("mes"+2 != "mes+2") THEN 'BAJA+2'
    -- si proximos 2 meses consecutivos activos:
    ELSE 'CONTINUA' END AS "clase_ternaria"
FROM (
    SELECT *,
    IF(foto_mes = 202105, 0, lead(mes,1,-1) OVER (PARTITION BY "numero_de_cliente" ORDER BY numero_de_cliente, mes)) AS 'mes+1',
    IF(foto_mes = 202105 OR foto_mes = 202104, 0,lead(mes,2,-1) OVER (PARTITION BY "numero_de_cliente" ORDER BY numero_de_cliente, mes)) AS 'mes+2',
    FROM
        (SELECT
            *,
            (foto_mes//100-2021)*12+foto_mes%100 as mes
        FROM competencia_01
        ORDER BY numero_de_cliente, foto_mes)
    ORDER BY numero_de_cliente, foto_mes);


,Success


In [17]:
%%sql
select 
    foto_mes,
    clase_ternaria,
    count(*) as cantidad
from proceso
group by foto_mes,clase_ternaria

,foto_mes,clase_ternaria,cantidad
0,202103,CONTINUA,161311
1,202104,CONTINUA,163637
2,202105,CONTINUA,164102
3,202103,BAJA+1,1050
4,202103,BAJA+2,963


In [20]:
%%sql
ALTER TABLE proceso
DROP COLUMN "mes+2";

,Success


## Ejercicio 1.1

* ¿Cuál es la nominalidad de cada clase?
* ¿Cuál es la proporción del target?

In [ ]:
%%sql
-- introduzca sus queries

Para guardar a un **.csv** simplemente debe ejecutar la siguiente sentencia

In [21]:
%%sql
COPY proceso TO '/home/tomi/Escritorio/Maestria/2 - DMEyF/datasets/competencia_01.csv' (FORMAT CSV, HEADER)

,Success


## Ejercicio 2 - Avanzado
Use ahora el archivo `ejercicio_target.csv` y calcule para todos los clientes en todos los periodos su **clase_ternaria** al mismo tiempo

In [44]:
%%sql
create or replace table competencia_01 as 
select 
    *
from read_csv_auto('/home/tomi/Escritorio/Maestria/2 - DMEyF/datasets/ejercicio_target.csv')

,Success


In [110]:
%sql select * from competencia_01 limit 5

,periodo,nombre
0,202110,Isaac Valverde
1,202110,Santiago Córdova Tercero
2,202110,Jerónimo Ocasio
3,202110,Isabel Terrazas Hijo
4,202110,Diego Caballero


In [ ]:
%sql select max(periodo) from competencia_01 limit 5

In [120]:
%%sql

SELECT nombre,periodo,meses_futuro_activo,
CASE WHEN meses_futuro_activo > 2 THEN 'CONTINUA'
    ELSE ('BAJA+' || meses_futuro_activo)
    END as clase_ternaria
FROM(
    SELECT nombre,periodo,
    SUM(aux) OVER (
        PARTITION BY "nombre"
        ORDER BY nombre, periodo
        ROWS BETWEEN 0 PRECEDING
            AND 2 FOLLOWING) AS meses_futuro_activo
    FROM 
        (SELECT 
            nombre,
            periodo,
            CASE 
                WHEN periodo = 202203 THEN 3 
                WHEN periodo = 202202 THEN 2 
                ELSE 1 END AS aux
        FROM competencia_01
        ORDER BY nombre, periodo)
    ORDER BY nombre, periodo
);

,nombre,periodo,meses_futuro_activo,clase_ternaria
0,Aarón Pantoja,202201,6.0,CONTINUA
1,Aarón Pantoja,202202,5.0,CONTINUA
2,Aarón Pantoja,202203,3.0,CONTINUA
3,Abril Palomino,202201,6.0,CONTINUA
4,Abril Palomino,202202,5.0,CONTINUA
...,...,...,...,...
734,Ximena Rael Segundo,202111,3.0,CONTINUA
735,Ximena Rael Segundo,202112,4.0,CONTINUA
736,Ximena Rael Segundo,202201,6.0,CONTINUA
737,Ximena Rael Segundo,202202,5.0,CONTINUA


In [115]:
%%sql
select 
    clase_ternaria
    , count(*) as cantidad
from proceso
group by clase_ternaria

,clase_ternaria,cantidad
0,CONTINUA,674
1,BAJA+1,36
2,BAJA+2,29
